https://medium.com/@mrunmayee.dhapre/book-recommendation-using-retrieval-augmented-generation-52965b71ed16

In [1]:
%pip install langchain langchain-community faiss-cpu langchain-huggingface

^C
Note: you may need to restart the kernel to use updated packages.


In [4]:
import pandas as pd
from langchain.vectorstores import FAISS
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.llms import HuggingFacePipeline
from langchain_huggingface import HuggingFaceEndpoint
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

In [5]:
df = pd.read_csv('Data/recipes_food_com_cleaned.csv')

In [ ]:
# subset = ['ID','Name', 'Description','Raw_Ingredients','Instructions']
# df = df[subset]
# df.head()

# df.to_csv('simple_recipes.csv', index=False)

In [ ]:
# load data
loader = CSVLoader(file_path='Data/recipes_food_com_cleaned.csv', encoding = "utf-8")
data = loader.load()

data = data[:100]

# transform data
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(data)

AttributeError: 'str' object has no attribute 'page_content'

In [ ]:
# embedding model
model_name = 'BAAI/bge-base-en-v1.5'

embeddings = HuggingFaceEmbeddings(model_name=model_name)

In [ ]:
# fill vector db
docsearch = FAISS.from_documents(texts, embeddings)
retriever=docsearch.as_retriever()

In [ ]:
# similarity search
ans = docsearch.similarity_search("beef tomato egg rice noodle")
print(ans[1].page_content)

In [ ]:
# define generation llm
# temperature defines how creative the model should be
llm = HuggingFaceEndpoint(repo_id = "microsoft/Phi-3-mini-4k-instruct", temperature=0.2)

In [ ]:
# ask a question to the llm
llm("Recommend me a recipe with beef, tomato, egg, and rice")

# experiencing hallucinations

In [ ]:
# creating a chat bot
template = '''
You are an assistant for recommending recipes. Find the first and only recipe with as much of the listed ingredients as possible.
Show the ingredients, steps, and nutrition.
Only use the recipes in the retrieved context to answer the question.

Question: {question}

Context: {context}

Answer:'''

custom_rag_prompt = PromptTemplate(input_variables=['question','context'], template=template)

qa_chain = LLMChain(
    llm=llm,
    prompt=custom_rag_prompt,
    output_parser=StrOutputParser()
)

In [ ]:
# ask a question to the chatbot
question = "Recommend me a recipe with potatoes and eggs"

# context is currently set to the first relevant doc
retrieved_doc = retriever.get_relevant_documents(question)[0]

response = qa_chain.invoke({
    "context": retrieved_doc.page_content,
    "question": question
})['text']

print(response)

In [ ]:
retrieved_doc.page_content